In [1]:
from openai import OpenAI
import json
import os
from tqdm import tqdm

# ================= CONFIGURACIÓN =================

MODEL = "gpt-4.1"          # recomendado para extracción de evidencia
CHUNK_SIZE = 30           # 20–50 es seguro
INPUT_GRAPH = "graph_input.json"
PROMPT_FILE = "prompt.txt"
OUTPUT_GRAPH = "graph_enriched.json"

client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])

# ================= CARGA =================

with open(INPUT_GRAPH, "r", encoding="utf-8") as f:
    graph = json.load(f)

with open(PROMPT_FILE, "r", encoding="utf-8") as f:
    system_prompt = f.read()

nodes = graph.get("nodes", [])
relationships = graph.get("relationships", [])

# Índice rápido de relaciones por nodo
rels_by_node = {}
for r in relationships:
    rels_by_node.setdefault(r["source"], []).append(r)
    rels_by_node.setdefault(r["target"], []).append(r)

# ================= UTILIDADES =================

def chunk_list(lst, size):
    for i in range(0, len(lst), size):
        yield lst[i:i+size]

enriched_nodes = []
enriched_relationships = []
processed_rel_ids = set()

# ================= PROCESAMIENTO =================

for node_chunk in tqdm(list(chunk_list(nodes, CHUNK_SIZE))):

    node_ids = {n["id"] for n in node_chunk}

    # relaciones asociadas a estos nodos
    rel_chunk = []
    for nid in node_ids:
        for r in rels_by_node.get(nid, []):
            # evitar duplicarlas muchas veces
            rid = (r["source"], r["target"], r["type"])
            if rid not in processed_rel_ids:
                rel_chunk.append(r)
                processed_rel_ids.add(rid)

    partial_graph = {
        "nodes": node_chunk,
        "relationships": rel_chunk
    }

    messages = [
        {"role": "system", "content": system_prompt},
        {
            "role": "user",
            "content": json.dumps(partial_graph, ensure_ascii=False)
        }
    ]

    response = client.chat.completions.create(
        model=MODEL,
        messages=messages,
        temperature=0.0,
        max_tokens=8000
    )

    raw_output = response.choices[0].message.content

    try:
        enriched_partial = json.loads(raw_output)
    except json.JSONDecodeError:
        # guardamos el output crudo para depurar
        with open("error_raw_output.txt", "w", encoding="utf-8") as f:
            f.write(raw_output)
        raise RuntimeError("❌ El modelo devolvió JSON inválido. Ver error_raw_output.txt")

    enriched_nodes.extend(enriched_partial.get("nodes", []))
    enriched_relationships.extend(enriched_partial.get("relationships", []))

# ================= GUARDADO FINAL =================

final_graph = {
    "nodes": enriched_nodes,
    "relationships": enriched_relationships
}

with open(OUTPUT_GRAPH, "w", encoding="utf-8") as f:
    json.dump(final_graph, f, indent=2, ensure_ascii=False)

print("✅ Grafo enriquecido guardado en:", OUTPUT_GRAPH)


KeyError: 'OPENAI_API_KEY'

1.  **Access Secrets Manager**: On the left-hand side panel of Colab, click on the "🔑 Secrets" icon.
2.  **Add a new secret**: Click on "Add new secret".
3.  **Name the secret**: In the "Name" field, type `OPENAI_API_KEY` (ensure it matches exactly, including capitalization).
4.  **Enter your API key**: In the "Value" field, paste your actual OpenAI API key.
5.  **Save the secret**: Make sure "Notebook access" is toggled ON for this secret.

Once you've added the secret, you can access it in your notebook using `userdata.get()` from the `google.colab` library. I will now generate a code cell to demonstrate this and fix the previous error.

In [2]:
import os
from google.colab import userdata

# Retrieve the API key from Colab's Secrets Manager
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

# You can then initialize your OpenAI client as before
# from openai import OpenAI
# client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])

print("OPENAI_API_KEY has been loaded from Colab Secrets.")

SecretNotFoundError: Secret OPENAI_API_KEY does not exist.

In [7]:
from openai import OpenAI
import json
import os
from tqdm import tqdm
from google.colab import userdata

# ================= CONFIGURACIÓN =================

MODEL = "gpt-4.1"          # recomendado para extracción de evidencia
CHUNK_SIZE = 30           # 20–50 es seguro
INPUT_GRAPH = "/content/final_graph_enriched_metadata_clean_260126_juani_sin snippet, justif y conf.json"
PROMPT_FILE = "/content/prompt agregar snippet, justif y confianza 26.01.txt"
OUTPUT_GRAPH = "graph_enriched.json"

# Retrieve the API key from Colab's Secrets Manager
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])

# ================= CARGA =================

with open(INPUT_GRAPH, "r", encoding="utf-8") as f:
    graph = json.load(f)

with open(PROMPT_FILE, "r", encoding="utf-8") as f:
    system_prompt = f.read()

nodes = graph.get("nodes", [])
relationships = graph.get("relationships", [])

# Índice rápido de relaciones por nodo
rels_by_node = {}
for r in relationships:
    rels_by_node.setdefault(r["source"], []).append(r)
    rels_by_node.setdefault(r["target"], []).append(r)

# ================= UTILIDADES =================

def chunk_list(lst, size):
    for i in range(0, len(lst), size):
        yield lst[i:i+size]

enriched_nodes = []
enriched_relationships = []
processed_rel_ids = set()

# ================= PROCESAMIENTO =================

for node_chunk in tqdm(list(chunk_list(nodes, CHUNK_SIZE))):

    node_ids = {n["id"] for n in node_chunk}

    # relaciones asociadas a estos nodos
    rel_chunk = []
    for nid in node_ids:
        for r in rels_by_node.get(nid, []):
            # evitar duplicarlas muchas veces
            rid = (r["source"], r["target"], r["type"])
            if rid not in processed_rel_ids:
                rel_chunk.append(r)
                processed_rel_ids.add(rid)

    partial_graph = {
        "nodes": node_chunk,
        "relationships": rel_chunk
    }

    messages = [
        {"role": "system", "content": system_prompt},
        {
            "role": "user",
            "content": json.dumps(partial_graph, ensure_ascii=False)
        }
    ]

    response = client.chat.completions.create(
        model=MODEL,
        messages=messages,
        temperature=0.0,
        max_tokens=8000
    )

    raw_output = response.choices[0].message.content

    try:
        enriched_partial = json.loads(raw_output)
    except json.JSONDecodeError:
        # guardamos el output crudo para depurar
        with open("error_raw_output.txt", "w", encoding="utf-8") as f:
            f.write(raw_output)
        raise RuntimeError("❌ El modelo devolvió JSON inválido. Ver error_raw_output.txt")

    enriched_nodes.extend(enriched_partial.get("nodes", []))
    enriched_relationships.extend(enriched_partial.get("relationships", []))

# ================= GUARDADO FINAL =================

final_graph = {
    "nodes": enriched_nodes,
    "relationships": enriched_relationships
}

with open(OUTPUT_GRAPH, "w", encoding="utf-8") as f:
    json.dump(final_graph, f, indent=2, ensure_ascii=False)

print("✅ Grafo enriquecido guardado en:", OUTPUT_GRAPH)

  0%|          | 0/5 [01:00<?, ?it/s]


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [8]:
pip install tenacity

In [2]:
import openai
from openai import OpenAI
import json
import os
from tqdm import tqdm
from google.colab import userdata
from tenacity import retry, wait_exponential, stop_after_attempt, retry_if_exception_type

# ================= CONFIGURACIÓN =================

MODEL = "gpt-4.1"          # recomendado para extracción de evidencia
CHUNK_SIZE = 30           # 20–50 es seguro
INPUT_GRAPH = "/content/final_graph_enriched_metadata_clean_260126_juani_sin snippet, justif y conf.json"
PROMPT_FILE = "/content/prompt agregar snippet, justif y confianza 26.01.txt"
OUTPUT_GRAPH = "graph_enriched.json"

# Retrieve the API key from Colab's Secrets Manager
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])

# ================= CARGA =================

with open(INPUT_GRAPH, "r", encoding="utf-8") as f:
    graph = json.load(f)

with open(PROMPT_FILE, "r", encoding="utf-8") as f:
    system_prompt = f.read()

nodes = graph.get("nodes", [])
relationships = graph.get("relationships", [])

# Índice rápido de relaciones por nodo
rels_by_node = {}
for r in relationships:
    rels_by_node.setdefault(r["source"], []).append(r)
    rels_by_node.setdefault(r["target"], []).append(r)

# ================= UTILIDADES =================

def chunk_list(lst, size):
    for i in range(0, len(lst), size):
        yield lst[i:i+size]

enriched_nodes = []
enriched_relationships = []
processed_rel_ids = set()

# Define the retry decorator
@retry(
    wait=wait_exponential(multiplier=1, min=4, max=60),
    stop=stop_after_attempt(6),
    retry=retry_if_exception_type(openai.APITimeoutError) | retry_if_exception_type(openai.APIConnectionError) | retry_if_exception_type(openai.RateLimitError)
)
def create_chat_completion_with_retry(model, messages, temperature, max_tokens):
    return client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens
    )

# ================= PROCESAMIENTO =================

for node_chunk in tqdm(list(chunk_list(nodes, CHUNK_SIZE))):

    node_ids = {n["id"] for n in node_chunk}

    # relaciones asociadas a estos nodos
    rel_chunk = []
    for nid in node_ids:
        for r in rels_by_node.get(nid, []):
            # evitar duplicarlas muchas veces
            rid = (r["source"], r["target"], r["type"])
            if rid not in processed_rel_ids:
                rel_chunk.append(r)
                processed_rel_ids.add(rid)

    partial_graph = {
        "nodes": node_chunk,
        "relationships": rel_chunk
    }

    messages = [
        {"role": "system", "content": system_prompt},
        {
            "role": "user",
            "content": json.dumps(partial_graph, ensure_ascii=False)
        }
    ]

    try:
        response = create_chat_completion_with_retry(
            model=MODEL,
            messages=messages,
            temperature=0.0,
            max_tokens=8000
        )
    except Exception as e:
        print(f"❌ Failed after multiple retries: {e}")
        raise # Re-raise the exception if all retries fail

    raw_output = response.choices[0].message.content

    try:
        enriched_partial = json.loads(raw_output)
    except json.JSONDecodeError:
        # guardamos el output crudo para depurar
        with open("error_raw_output.txt", "w", encoding="utf-8") as f:
            f.write(raw_output)
        raise RuntimeError("❌ El modelo devolvió JSON inválido. Ver error_raw_output.txt")

    enriched_nodes.extend(enriched_partial.get("nodes", []))
    enriched_relationships.extend(enriched_partial.get("relationships", []))

# ================= GUARDADO FINAL =================

final_graph = {
    "nodes": enriched_nodes,
    "relationships": enriched_relationships
}

with open(OUTPUT_GRAPH, "w", encoding="utf-8") as f:
    json.dump(final_graph, f, indent=2, ensure_ascii=False)

print("✅ Grafo enriquecido guardado en:", OUTPUT_GRAPH)

  0%|          | 0/5 [03:10<?, ?it/s]

❌ Failed after multiple retries: RetryError[<Future at 0x7a37dc5cf650 state=finished raised RateLimitError>]


RetryError: RetryError[<Future at 0x7a37dc5cf650 state=finished raised RateLimitError>]